<a href="https://colab.research.google.com/github/DCI-alxogm/ml2023-starmath4/blob/main/Sep29-30/Clase_30_de_septiembre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
import numpy as np
import random

import numpy as np
import random

def modelo_polinomio(punto,coefs): #Evaluar el polinomio en un punto
  sum=0
  for i in range (0,len(coefs)):
    sum+=coefs[i]*((punto)**i )
  return sum

def log_likelihood(x,y,coefs,sig):
  sum=0
  for i in range(len(x)):
    sum+=(((y[i]-modelo_polinomio(x[i],coefs))**2)-(2*sig))
  return sum

def log_prior(x,prop,int_w0,int_w1): #La propuesta debe caer en ambos prior
  if (int_w0[0]<prop[0] and prop[0]<int_w0[1]) and (int_w1[0]<prop[1] and prop[1]<int_w1[1]):
    return np.log(1)
  else:
    return np.log(0)

def log_post(x,y,coefs,sig,int_w0,int_w1):
  return log_likelihood(x,y,coefs,sig)+log_prior(x,coefs,int_w0,int_w1)

In [82]:
data=np.loadtxt("olympic100m_men.txt",delimiter=",").T #Delimitados por coma ya que es archivo separado por comas, .T indica la transpuesta
x=data[0][:] #De data, el primer arreglo es de años
y=data[1][:] #De data, el segundo arreglo es de los records

#Cadena Monte Carlo Markov
#Algoritmo de Metropolis

#Parametros propuestos
t_in=np.array([1,1])
sigma=2
n_sample=10

#Genero las distribuciones gausianas
gaussiana_0=np.random.normal(t_in[0],sigma,100)
gaussiana_1=np.random.normal(t_in[1],sigma,100)

t_prop=np.array([gaussiana_0[random.randint(0,99)],gaussiana_1[random.randint(0,99)]]) #La propuesta será algún punto que pertenezca a la gaussiana generado por los puntos de t
#Hay que normalizar los datos, de ser asi no importa mucho si las sigmas son diferentes o no

t_aceptadas=np.zeros((n_sample,2)) #t aceptadas
t_likes=np.zeros(n_sample) #guardo el likelihood
int_w0=[30,40]
int_w1=[0,-1]

for i in range(n_sample):
  log_post_t_in=log_post(x,y,t_in,sigma,int_w0,int_w1) #Calculo el log posterior de t inicial
  log_post_t_prop=log_post(x,y,t_prop,sigma,int_w0,int_w1) #Calculo el log posterior de t propuesto
  if log_post_t_prop>log_post_t_in: #Comparación de los posterior
    t_aceptadas[i,:]=t_prop #Guardo t prop en un arreglo de las propuesta
    t_in=t_prop #Remplazo t_in por t_prop
    t_likes=log_likelihood(x,y,t_in,sigma)
  else:
    rand=(random.randint(1,100))/100 #Genero un numero aleatorio entre 0 y 1
    if (log_post_t_prop-log_post_t_in)>np.log(rand): #Radio del logaritmo posterior y el inicial, para logaritmos se convierte en una resta y por eso le aplicamos logaritmo al numero aleatorio
      t_aceptadas[i,:]=t_prop #Guardo t prop en un arreglo de las propuesta
      t_in=t_prop #Remplazo t_in por t_pro
      t_likes=log_likelihood(x,y,t_in,sigma)
    else:
      t_aceptadas[i,:]=t_in #Guardo t in en un arreglo de las propuesta
      t_likes=log_likelihood(x,y,t_in,sigma)

<ipython-input-69-b20c3c8955dd>:23: RuntimeWarning: divide by zero encountered in log
  return np.log(0)
<ipython-input-82-0ab47374293d>:34: RuntimeWarning: invalid value encountered in double_scalars
  if (log_post_t_prop-log_post_t_in)>np.log(rand): #Radio del logaritmo posterior y el inicial, para logaritmos se convierte en una resta y por eso le aplicamos logaritmo al numero aleatorio
